In [9]:
import h5py
import sys
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
sys.path.append('/home/ztang/multitask_RNA/data_generation')
import utils
import numpy as np
import nucleotide_transformer
import haiku as hk
import jax
import jax.numpy as jnp
from nucleotide_transformer.pretrained import get_pretrained_model
from tqdm import tqdm
model_name = '` `'

In [10]:
data_file = h5py.File('/home/ztang/multitask_RNA/data/lenti_MPRA/HepG2_data.h5', 'r')
sequence = data_file['seq'][()]
target = data_file['mean'][()]

In [11]:
parameters, forward_fn, tokenizer, config = get_pretrained_model(
    model_name=model_name,
    mixed_precision=False,
    embeddings_layers_to_save=(32,),
    attention_maps_to_save=(),
    max_positions=41,
)
forward_fn = hk.transform(forward_fn)

In [12]:
N,  = sequence.shape
seq_pair = []
for i in tqdm(range(N)):
    seq = sequence[i].decode()
    token_out = tokenizer.batch_tokenize([seq])
    token_id = [b[1] for b in token_out]
    seq_pair.append(np.squeeze(token_id))
seq_pair = jnp.asarray(seq_pair,dtype=jnp.int32)

100%|██████████| 139877/139877 [00:03<00:00, 46142.42it/s]


In [13]:
#get embedding per input sequence
batch_size = 256
lenti_embed = []
random_key = jax.random.PRNGKey(0)
for i in tqdm(range(0, N, batch_size)):
    seq_batch = seq_pair[i:i+batch_size]
    outs = forward_fn.apply(parameters, random_key, seq_batch)
    lenti_embed.append(outs['embeddings_32'][0])

100%|██████████| 547/547 [11:02<00:00,  1.21s/it]


In [14]:
out_embed = h5py.File('/home/ztang/multitask_RNA/data/lenti_MPRA/HepG2_data_embed.h5', 'w')
out_embed.create_dataset('embed', data=np.array(lenti_embed))
out_embed.create_dataset('target', data=np.array(target))

<HDF5 dataset "target": shape (139877,), type "<f8">

In [8]:
out_embed.close()